# Chat Agent Executor with Anthropic


In this example we will build a chat executor that uses tool calling and the prebuilt ToolNode with Anthropic.

In [1]:
from langgraph_sdk import get_client

In [2]:
client = get_client()

In [3]:
assistant = await client.assistants.create(graph_id="agent")
assistant

{'assistant_id': 'c9220253-f609-42f2-85dc-9ca03c3c8b04',
 'graph_id': 'agent',
 'config': {},
 'created_at': '2024-05-02T18:55:06.193876+00:00',
 'updated_at': '2024-05-02T18:55:06.193876+00:00',
 'metadata': {}}

In [4]:
thread = await client.threads.create()
thread

{'thread_id': 'ad3fca2d-eae6-436b-8dc6-6124902a380e',
 'created_at': '2024-05-02T18:55:06.509718+00:00',
 'updated_at': '2024-05-02T18:55:06.509718+00:00',
 'metadata': {}}

In [5]:
runs = await client.runs.list(thread['thread_id'])
runs

[]

In [6]:
input = {"messages": [{"role": "human", "content": "whats the weather in la"}]}
async for chunk in client.runs.stream(thread['thread_id'], "agent", input=input):
    print(chunk)

StreamPart(event='metadata', data={'run_id': '301982c4-b1a3-4ef1-8e37-b6becf44f0c9'})
StreamPart(event='data', data={'messages': [{'role': 'human', 'content': 'whats the weather in la'}]})
StreamPart(event='data', data={'messages': [{'role': 'human', 'content': 'whats the weather in la'}, {'content': [{'id': 'toolu_01LmrgVPgScZyrfdnkeGqqD3', 'input': {'query': 'weather in los angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': 'run-5303a4b2-1753-4eee-aad8-d4282cd2f931', 'example': False, 'tool_calls': [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in los angeles'}, 'id': 'toolu_01LmrgVPgScZyrfdnkeGqqD3'}], 'invalid_tool_calls': []}]})
StreamPart(event='data', data={'messages': [{'role': 'human', 'content': 'whats the weather in la'}, {'content': [{'id': 'toolu_01LmrgVPgScZyrfdnkeGqqD3', 'input': {'query': 'weather in los angeles'}, 'name': 'tavily_search_results_js

In [7]:
input = {"messages": [{"role": "human", "content": "and in sf"}]}
run = await client.runs.create(thread['thread_id'], assistant["assistant_id"], input=input)
run

{'run_id': '9a4e328d-b095-4d33-9ef9-23c2524fea8d',
 'thread_id': 'ad3fca2d-eae6-436b-8dc6-6124902a380e',
 'assistant_id': 'c9220253-f609-42f2-85dc-9ca03c3c8b04',
 'created_at': '2024-05-02T18:55:18.308103+00:00',
 'updated_at': '2024-05-02T18:55:18.308103+00:00',
 'status': 'pending',
 'metadata': {}}

In [15]:
await client.runs.get(thread['thread_id'], run['run_id'])

{'run_id': '9a4e328d-b095-4d33-9ef9-23c2524fea8d',
 'thread_id': 'ad3fca2d-eae6-436b-8dc6-6124902a380e',
 'assistant_id': 'c9220253-f609-42f2-85dc-9ca03c3c8b04',
 'created_at': '2024-05-02T18:55:18.308103+00:00',
 'updated_at': '2024-05-02T18:55:18.308103+00:00',
 'status': 'success',
 'metadata': {}}

In [14]:
await client.runs.list_events(thread['thread_id'], run['run_id'])

[{'event_id': 'e7af16e6-99ca-43b3-ae83-6129b7c29412',
  'run_id': '9a4e328d-b095-4d33-9ef9-23c2524fea8d',
  'received_at': '2024-05-02T18:55:26.148266+00:00',
  'span_id': '9a4e328d-b095-4d33-9ef9-23c2524fea8d',
  'event': 'on_chain_end',
  'name': 'LangGraph',
  'data': {'output': {'messages': [{'role': 'human',
      'content': 'whats the weather in la'},
     {'id': 'run-5303a4b2-1753-4eee-aad8-d4282cd2f931',
      'name': None,
      'type': 'ai',
      'content': [{'id': 'toolu_01LmrgVPgScZyrfdnkeGqqD3',
        'name': 'tavily_search_results_json',
        'type': 'tool_use',
        'input': {'query': 'weather in los angeles'}}],
      'example': False,
      'tool_calls': [{'id': 'toolu_01LmrgVPgScZyrfdnkeGqqD3',
        'args': {'query': 'weather in los angeles'},
        'name': 'tavily_search_results_json'}],
      'additional_kwargs': {},
      'response_metadata': {},
      'invalid_tool_calls': []},
     {'id': None,
      'name': 'tavily_search_results_json',
      'type